# membrane-toolkit data aggregation demo

This notebook will demonstrate some of the capabilities of membrane-toolkit's core functions.

## Setup
---

In this example, we will use the `PermselectivityDrone` to parse membrane potential data from a folder full of spreadsheets. The `Drone` will create a pandas `DataFrame` out of the data. We'll then use that `DataFrame` calculate the ideal membrane potential and the apparent permselectivity using the `nernst_potential` and `apparent_permselectivity` functions from `membrane_toolkit.core`

In [1]:
from membrane_toolkit.pipeline.drones import PermselectivityDrone
from membrane_toolkit.core import apparent_permselectivity, nernst_potential

/home/ryan/anaconda3/envs/wsl2/lib/python3.8/site-packages/maggma/utils.py:20: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


**NOTE:** when importing, you must use and underscore instead of a hyphen in 'membrane_toolkit'

## Main code
---

### Create the drone

The first step to parsing data with membrane-toolkit is to create a `Drone` that knows the location of your data files. The Drone will loop through every file in the directory and parse their data Each `Drone` in membrane-toolkit has an associated default spreadsheet template and a corresponding configuration file. Both of these are easy to customize. The image below shows an example of how specific parts of the template spreadsheet map to values in the configuration file.

![test](drone_template_config.png)

For this example, we will create a `PermselectivityDrone` because we have collected membrane potential data for permselectivity calculations. All we need to do to create the `Drone` is supply the location of our data folder:

In [2]:
from pathlib import Path
data_folder = Path.cwd() / 'permselectivity_data'
drone = PermselectivityDrone(data_folder)

### Run the drone

Next, simply call `drone.run()` and all the files (6 in this case) will be parsed.

In [3]:
drone.run()

2020-05-10 18:45:11,844 - PermselectivityDrone - DEBUG - Starting get_items in PermselectivityDrone Builder



2020-05-10 18:45:11,905 - PermselectivityDrone - INFO - Processing batch of 1000 items
2020-05-10 18:45:12,054 - PermselectivityDrone - DEBUG - Updating 6 items


### Examine the data

The drone populates a [`Store`](https://materialsproject.github.io/maggma/concepts/) object with all the parsed data. Stores provide a consistent interface for adding data to several different database endpoints. In membrane-toolkit, the default `Store` adds data to a pandas dataframe, which can be viewed by calling `drone.store.as_df()`

In [4]:
df = drone.store.as_df()
df

,date,person,sample_id,membrane,temperature,electrode_offset,salt_1,conc_1,salt_2,conc_2,membrane_potential,notes,last_updated,documents,record_key
0,2019-12-10,John Smith,AMX-0.5-2,AMX,24.3,-1.0,NaCl,0.5,NaCl,0.1,35.0,None,2020-05-10 18:45:11.865,[{'path': '/mnt/d/UNC Drive/pymemsci/examples/...,permselectivity_1.xlsx
1,2019-12-11,John Smith,AMX-0.5-3,AMX,24.7,-0.8,NaCl,0.5,NaCl,0.1,34.9,None,2020-05-10 18:45:11.868,[{'path': '/mnt/d/UNC Drive/pymemsci/examples/...,permselectivity_2.xlsx
2,2019-12-08,John Smith,AMX-0.5-1,AMX,24.9,-0.9,NaCl,0.5,NaCl,0.1,35.2,None,2020-05-10 18:45:11.870,[{'path': '/mnt/d/UNC Drive/pymemsci/examples/...,permselectivity_3.xlsx
3,2020-01-14,Jane Doe,CMX-5,CMX,23.0,-0.2,NaCl,0.5,NaCl,0.1,35.0,None,2020-05-10 18:45:11.872,[{'path': '/mnt/d/UNC Drive/pymemsci/examples/...,permselectivity_4.xlsx
4,2020-01-16,Jane Doe,CMX-7,CMX,23.8,-0.4,NaCl,0.5,NaCl,0.1,35.8,None,2020-05-10 18:45:11.875,[{'path': '/mnt/d/UNC Drive/pymemsci/examples/...,permselectivity_5.xlsx
5,2020-01-17,Jane Doe,CMX-8,CMX,24.0,-0.5,NaCl,0.5,NaCl,0.1,35.4,None,2020-05-10 18:45:11.877,[{'path': '/mnt/d/UNC Drive/pymemsci/examples/...,permselectivity_6.xlsx


### Filter the data

You'll notice that in addition to the fields from the spreadsheet data, the drone has created a few extra fields like `record_key` and `documents`, and some of the data like the person running the experiment may not be relevant to the analysis we want to do. Pandas dataframes make it easy to filter the data so you only see what you need. For example, if we only want to see the membrane type, temperature, date, and sample id:

In [5]:
df.xs((["sample_id", "membrane","date","temperature"]),axis=1)

,sample_id,membrane,date,temperature
0,AMX-0.5-2,AMX,2019-12-10,24.3
1,AMX-0.5-3,AMX,2019-12-11,24.7
2,AMX-0.5-1,AMX,2019-12-08,24.9
3,CMX-5,CMX,2020-01-14,23.0
4,CMX-7,CMX,2020-01-16,23.8
5,CMX-8,CMX,2020-01-17,24.0


### Make some calculations

Now that we have all the experimental data in one place, we can calculate the ideal membrane potential and the apparent permselectivity. First, we'll use the [`nerst_potential`]() core function to calculate the ideal membrane potential based on data in the `conc_1`, `conc_2`, and `temperature` columns. To do this, we'll use the pandas [`apply`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html) method.

In [6]:
df["ideal_potential"] = df.apply(lambda row: nernst_potential(row["conc_1"], row["conc_2"], 1, row["temperature"]) * 1000, axis=1)
df

,date,person,sample_id,membrane,temperature,electrode_offset,salt_1,conc_1,salt_2,conc_2,membrane_potential,notes,last_updated,documents,record_key,ideal_potential
0,2019-12-10,John Smith,AMX-0.5-2,AMX,24.3,-1.0,NaCl,0.5,NaCl,0.1,35.0,None,2020-05-10 18:45:11.865,[{'path': '/mnt/d/UNC Drive/pymemsci/examples/...,permselectivity_1.xlsx,41.251374
1,2019-12-11,John Smith,AMX-0.5-3,AMX,24.7,-0.8,NaCl,0.5,NaCl,0.1,34.9,None,2020-05-10 18:45:11.868,[{'path': '/mnt/d/UNC Drive/pymemsci/examples/...,permselectivity_2.xlsx,41.306847
2,2019-12-08,John Smith,AMX-0.5-1,AMX,24.9,-0.9,NaCl,0.5,NaCl,0.1,35.2,None,2020-05-10 18:45:11.870,[{'path': '/mnt/d/UNC Drive/pymemsci/examples/...,permselectivity_3.xlsx,41.334584
3,2020-01-14,Jane Doe,CMX-5,CMX,23.0,-0.2,NaCl,0.5,NaCl,0.1,35.0,None,2020-05-10 18:45:11.872,[{'path': '/mnt/d/UNC Drive/pymemsci/examples/...,permselectivity_4.xlsx,41.071086
4,2020-01-16,Jane Doe,CMX-7,CMX,23.8,-0.4,NaCl,0.5,NaCl,0.1,35.8,None,2020-05-10 18:45:11.875,[{'path': '/mnt/d/UNC Drive/pymemsci/examples/...,permselectivity_5.xlsx,41.182032
5,2020-01-17,Jane Doe,CMX-8,CMX,24.0,-0.5,NaCl,0.5,NaCl,0.1,35.4,None,2020-05-10 18:45:11.877,[{'path': '/mnt/d/UNC Drive/pymemsci/examples/...,permselectivity_6.xlsx,41.209769


Next, we use the `apparent_permselectivity` core function to compute the membrane permselectivity. In this case, we can just pass the dataframe columns as arguments to the function directly (no need for `apply`).

In [7]:
df["permselectivity"] = apparent_permselectivity(df["membrane_potential"], df["ideal_potential"])

Now we can create another cross-section view that includes the result.

In [8]:
df.xs((["sample_id","date","membrane","conc_1","conc_2","temperature","permselectivity"]),axis=1)

,sample_id,date,membrane,conc_1,conc_2,temperature,permselectivity
0,AMX-0.5-2,2019-12-10,AMX,0.5,0.1,24.3,0.848457
1,AMX-0.5-3,2019-12-11,AMX,0.5,0.1,24.7,0.844896
2,AMX-0.5-1,2019-12-08,AMX,0.5,0.1,24.9,0.851587
3,CMX-5,2020-01-14,CMX,0.5,0.1,23.0,0.852181
4,CMX-7,2020-01-16,CMX,0.5,0.1,23.8,0.869311
5,CMX-8,2020-01-17,CMX,0.5,0.1,24.0,0.859020


### Generate summary statistics

The `.groupby` method makes it easy to generate summary statics like mean or standard deviation for each membrane type.

In [9]:
df.groupby('membrane').mean()

,temperature,electrode_offset,conc_1,conc_2,membrane_potential,ideal_potential,permselectivity
membrane,,,,,,,
AMX,24.633333,-0.900000,0.5,0.1,35.033333,41.297602,0.848313
CMX,23.600000,-0.366667,0.5,0.1,35.400000,41.154296,0.860171


In [10]:
df.groupby('membrane').std()

,temperature,electrode_offset,conc_1,conc_2,membrane_potential,ideal_potential,permselectivity
membrane,,,,,,,
AMX,0.305505,0.100000,0.0,0.0,0.152753,0.042368,0.003348
CMX,0.529150,0.152753,0.0,0.0,0.400000,0.073384,0.008623
